In [1]:
import requests
import bs4
import pandas as pd


URL = "http://escalade.orsay.free.fr/matos/Index_voies.php"

r = requests.get(URL)
soup = bs4.BeautifulSoup(r.text, "lxml")

In [2]:
table = soup.select_one("#matosTable > table")
df = pd.read_html(str(table), header=0, parse_dates=['Date'])[0]

In [3]:
df

,Nom,Cotation,Couleur,Relais,Commentaire,Ouvreur,Date
0,Le Pas,4a,vieilles grises,1 (mur gris),R1,NaN,2007-01-01
1,"Ce n'est pas dur, c'est juste impressionnant",6a,saumons,2 (mur gris),R2. Départ sous R1,Stéphane M,2009-11-01
2,NaN,6b,oranges marbrées,2 (mur gris),R2. Départ sous R3,Sylvain,2007-01-01
3,NaN,6a,vieilles prises,3 (mur gris),R3. Avec 1ère réglette du mur. Pas final 6a+,José,2008-09-01
4,NaN,5b+,jaunes,3 (mur gris),R3,NaN,2007-01-01
...,...,...,...,...,...,...,...
143,NaN,5c,jaunes,24 (nouveau mur),"Avec arête, module, carré Entreprises",Estelle et Thibault,2018-01-01
144,Gecko si rando,6b+,saumons,24 (nouveau mur),Départ assis. Sans module. Avec arête et carré...,Aurélien,2017-10-01
145,Second macaque aveugle,6a,bicolores,24 (nouveau mur),"Zéro prise ! Départ assis, arête D et réglette...",Sylvain,2018-06-01
146,"Droite tâte, gauche pioche",6a+,vertes,24 (nouveau mur),"Micro prises. Départ assis, avec arête D",Aurélien,2019-01-01


In [4]:
# import sys
# dump the dataframe to json to stdout
# df.to_json(sys.stdout, orient="records", indent=4)

In [5]:
import re
from itertools import chain

def get_flag(flag, string):
    if flag not in string.lower():
        return False
    
    string = re.sub(r"\b(D|G)\b\.(?!= [A-Z])", r"\1", string)
    
    for s in re.findall(r"\([^)]*\)", string):
        if flag in s.lower():
            return get_flag(flag, s[1:-1])

    string = re.sub(r"\([^)]*\)", "", string)
    
    modifiers = {
        'positive': { 'avec' }, 
        'negative': { 'sans', 'ni', 'pas' },
        'location': { 'à' }
    }

    pattern = re.compile(f"(?:.*)({'|'.join(chain(*modifiers.values()))})(?=[^.]*{flag})", re.IGNORECASE)
    match = pattern.search(string)

    if match is None:
        return True

    exclude = chain.from_iterable(modifiers[key] for key in modifiers if key != 'positive')
    
    return match.group(1).lower() not in exclude


In [6]:
from functools import partial

def get_flags(comment: str):
    return {
        "Départ assis": partial(get_flag, "départ assis")(comment),
        "Module": partial(get_flag, "module")(comment),
        "Arête": partial(get_flag, "arête")(comment),
    }

In [7]:
assert(get_flags("Bleu clair. Départ assis sous R7") == { "Départ assis": True, "Module": False, "Arête": False })
assert(get_flags("2 variantes, fin R8 ou R10") == { "Départ assis": False, "Module": False, "Arête": False })
assert(get_flags("3 prises ! Sans module. Avec dièdre et arête.") == { "Départ assis": False, "Module": False, "Arête": True })
assert(get_flags("5a dalle, 5c vertical, cotations à confirmer, prises à grosses vis. Dalle faisable sans mains") == { "Départ assis": False, "Module": False, "Arête": False })
assert(get_flags("5a+ dalle, 5c+ vertical, cotations à confirmer. Départ assis") == { "Départ assis": True, "Module": False, "Arête": False })
assert(get_flags("7b?") == { "Départ assis": False, "Module": False, "Arête": False })
assert(get_flags("Arêtes autorisées, déborde sous R17") == { "Départ assis": False, "Module": False, "Arête": True })
assert(get_flags("Avec 1ère colonnette. Sans les autres.") == { "Départ assis": False, "Module": False, "Arête": False })
assert(get_flags("Avec arête") == { "Départ assis": False, "Module": False, "Arête": True })
assert(get_flags("Avec arête et module.") == { "Départ assis": False, "Module": True, "Arête": True })
assert(get_flags("Avec arête, module, carré Entreprises") == { "Départ assis": False,"Module" :True,"Arête" :True})
assert(get_flags("Avec carré Entreprises. Sans module,sans arête. Cot. à confirmer.") == {"Départ assis" :False,"Module" :False,"Arête" :False})
assert(get_flags("Avec inserts et arêtes") == {"Départ assis" :False,"Module" :False,"Arête" :True})
assert(get_flags("Avec module (5b+ sans)") == {"Départ assis" :False,"Module" :True,"Arête" :False})
assert(get_flags("Avec module et arête à D. Départ assis.") == {"Départ assis" :True,"Module" :True,"Arête" :True})
assert(get_flags("Avec module. Pas final 5c") == {"Départ assis" :False,"Module" :True,"Arête" :False})
assert(get_flags("Avec module. Pas final en 5b+") == {"Départ assis" :False,"Module" :True,"Arête" :False})
assert(get_flags("Avec module. Sans arête") == {"Départ assis" :False,"Module" :True,"Arête" :False})
assert(get_flags("Avec module. Sans arête.") == {"Départ assis" :False,"Module" :True,"Arête" :False})
assert(get_flags("Avec module. Technique.") == {"Départ assis" :False,"Module" :True,"Arête" :False})
assert(get_flags("Avec mur G (sans les prises), départ assis dos au mur.") == {"Départ assis" :True,"Module" :False,"Arête" :False})
assert(get_flags("Avec mur G (avec module), départ assis dos au mur.") == {"Départ assis" :True,"Module" :True,"Arête" :False})
assert(get_flags("Départ assis sous R10. Fin en R8. Sans module. Trois prises à G. de l'arête.") == {"Départ assis" :True,"Module" :False,"Arête" :False})
assert(get_flags("Avec module et arête à D. Départ assis.") == {"Départ assis" :True,"Module" :True,"Arête" :True})